<h1>PANDAS</h1>
"#"를 앞에 여러 개 붙여서 header 크기 조절 가능. 

In [32]:
import pandas as pd
import numpy as np
import pickle

In [2]:
df = pd.read_csv('test.csv', delimiter=',', header=None, names=['col1', 'col2', 'col3'])

In [6]:
type(df.col1) # df는 Series의 모음. 

pandas.core.series.Series

In [5]:
df

,col1,col2,col3
0,John Snow,23,male
1,Mao,56,male
2,Merkel,60,female
3,Trump,60,male


In [7]:
df[['col1', 'col2']]

,col1,col2
0,John Snow,23
1,Mao,56
2,Merkel,60
3,Trump,60


dictionary가 들어있는 list로도 pd.DataFrame()안에 넣어 df로 만들 수 있음. 

In [4]:
df.to_csv('test2.csv', index=True, header=True, na_rep = '-') # True가 default. 엑셀에서 보려고 할 땐 Index = False가 유용할 듯. na_rap은 NaN 대신 represent 하는 것. 
df.to_pickle('test2.p') # 장점: 용량작고, 빠름. 단점: 사람이 읽기 힘듦, 범용성 떨어짐.  

#### loc & iloc

In [8]:
df.loc[[0,2]] # location

,col1,col2,col3
0,John Snow,23,male
2,Merkel,60,female


In [16]:
df[(df.col2 > 50) & (df.col3 == ' male')] # and 대신 &. 

,col1,col2,col3
1,Mao,56,male
3,Trump,60,male


In [20]:
df.iloc[0:2, 0:1 ] # index location [row얼마나, column얼마나 ]

,col1
0,John Snow
1,Mao


#### filter, drop

In [25]:
df.filter(like='c', axis=1) # column name에 c가 들어간 것. 
# TypeError: Keyword arguments `items`, `like`, or `regex` are mutually exclusive

,col1,col2,col3
0,John Snow,23,male
1,Mao,56,male
2,Merkel,60,female
3,Trump,60,male


In [29]:
df.drop(['col3'], axis=1, inplace=False) # Inplace=True 되면 df 자체가 바뀜. axis=0이 default. 열 지울 땐 1 

,col1,col2
0,John Snow,23
1,Mao,56
2,Merkel,60
3,Trump,60


In [30]:
df.drop(df.index[[0,2]]) # index를 이용해 삭제할 때. 

,col1,col2,col3
1,Mao,56,male
3,Trump,60,male


In [36]:
df['is_president'] = np.where(df['col1'] != 'John Snow', 'yes', 'no') # 쉽게 조건을 걸어 새 열을 생성 가능. 

In [37]:
df

,col1,col2,col3,is_president
0,John Snow,23,male,no
1,Mao,56,male,yes
2,Merkel,60,female,yes
3,Trump,60,male,yes


#### group by

In [40]:
groups_sex = df.groupby('col3')

In [41]:
groups_sex.groups

{' female': Int64Index([2], dtype='int64'),
 ' male': Int64Index([0, 1, 3], dtype='int64')}

In [42]:
# 나중에 reset_index도 봐보기. 

In [45]:
df.duplicated(['col1'], keep='first') # 행마다, duplicate 있는지 확인. 어떤 행을 중복판별에 쓸 지 고를 수 있음. keep = first가 default. last로 바꿀 수도. (어떤거 남길지)
df.drop_duplicates() # 중복 제거. 

,col1,col2,col3,is_president
0,John Snow,23,male,no
1,Mao,56,male,yes
2,Merkel,60,female,yes
3,Trump,60,male,yes


In [46]:
df.shape

(4, 4)

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
col1            4 non-null object
col2            4 non-null int64
col3            4 non-null object
is_president    4 non-null object
dtypes: int64(1), object(3)
memory usage: 208.0+ bytes


#### missing data

In [51]:
df.isna() # == df.inull()

,col1,col2,col3,is_president
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False


In [61]:
df.col1 = df.col1.fillna(df.groupby('is_president')['col2'].transform('median')) # missing data 채워주기. group by 하여 각각 group의 median으로 채우는 것 가능. 

In [56]:
df

,col1,col2,col3,is_president
0,John Snow,23,male,no
1,Mao,56,male,yes
2,Merkel,60,female,yes
3,Trump,60,male,yes


#### apply

In [67]:
def age_remaining(year, is_dead):
    if is_dead == 1:
        return (100 - year)
    else:
        return (150 - year)


In [68]:
df['life_remaining'] = df.col2.apply(age_remaining, is_dead=0) # 여러 parameter을 이렇게 보낼 수 있다. 

In [69]:
df

,col1,col2,col3,is_president,life_remaining
0,John Snow,23,male,no,127
1,Mao,56,male,yes,94
2,Merkel,60,female,yes,90
3,Trump,60,male,yes,90


#### map

In [77]:
df.is_president = df.is_president.map({"yes":1, "no":0}) # applymap은 모든 column에 적용할 때 사용한다. (df.all.map 한다고 되지 않음.)

In [71]:
df

,col1,col2,col3,is_president,life_remaining
0,John Snow,23,male,0,127
1,Mao,56,male,1,94
2,Merkel,60,female,1,90
3,Trump,60,male,1,90


#### unique

In [78]:
df.col3.unique()

array([' male', ' female'], dtype=object)

In [79]:
df.col3.value_counts()

 male      3
 female    1
Name: col3, dtype: int64

#### dataframe 합치기

result = pd.concat([df1, df2], ignore_index=True) # append랑 기본적으로 같음. axis=1해줘야 column에 붙임. 
result = df1.append(df2, ignore_inde=True)

